In [18]:
from scipy import interpolate
import matplotlib.pyplot as plt

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pykalman import KalmanFilter

import torch
import torch.nn as nn
import torch.nn.functional as F

import copy


def mlesigma(f_name):
    # df = pd.read_excel('I-V_data_25min_AAO_10min_Pb_ED_1h_180C_MAI_memory_8V_2.xlsx') #0.55
    # df = pd.read_excel('I-V_data_30min_AAO_5min second etch_15min_Pb_ED_3h_180C_MAI_200nm_Ag_memory_6V.xlsx') #0.33
    # df = pd.read_excel('I-V_data_25min_AAO_10min_Pb_ED_1h_180C_MAI_memory_8V.xlsx') #0.14

    # data = pd.read_excel('../hardware_noise/I-V_data_25min_AAO_10min_Pb_ED_1h_180C_MAI_memory_8V.xlsx') #0.14
    # data = pd.read_excel('../hardware_noise/I-V_data_30min_AAO_5min second etch_15min_Pb_ED_3h_180C_MAI_200nm_Ag_memory_6V.xlsx') #0.33
    data = pd.read_excel(f_name, engine='openpyxl') #0.55

    conductance = data.drop(['voltage'], axis=1).div(data['voltage'], axis=0)

    conductance = conductance[(conductance!=np.inf).all(axis=1)]

    # (conductance.div(conductance.mean(axis=1), axis=0)<0).all(axis=0)


    from scipy.stats import norm
    import matplotlib.pyplot as plt

    conductance_by_mean = np.array(conductance.div(conductance.mean(axis=1), axis=0)).reshape(1, -1)[0]
    samples = np.log(conductance_by_mean)
    samples_fine = np.delete(conductance_by_mean, np.isfinite(samples)==False)

    # print(norm.fit(samples_fine))  # 返回极大似然估计
    # print(norm.fit(samples_fine, floc=0))  # 返回极大似然估计 固定均值为 0
    
    return norm.fit(samples_fine)[1]


# print(mlesigma('I-V_data_25min_AAO_10min_Pb_ED_1h_180C_MAI_memory_8V_2.xlsx')) #0.55


def weights_mapping(f_name, model, device='cuda'):
    """
    Map target weight to true weight. 
    The difference between the target weight and the true weight is due to two factors:
    1.Random heat noises
    2.Non-monotonic characteristics of conductance-Q curves
    """
    
    '''
    X ~ logNorm(a, b)  # m, v 是对数正态分布本身的均值和方差
    # 这说的是 X 是一个值永远为正的随机变量，只有取对数之后才是正态分布
    即 logX ~ Norm(mu, sigma)

    换算关系：
    m = exp(mu + sigma^2/2)  # 注意到 m 是 >0 的
    v = exp(2*mu + sigma^2) * exp(sigma^2 - 1)  

    因此，一个随机变量 Y 如果服从正态分布，即 Y ~ Norm(mu, sigma)
    就一定可以有一个 X = exp(Y), 使得 Y = logX ~ Norm(mu, sigma)

    所以，一个正态分布随机变量取指数得到的变量就服从对数正态分布

    具体到上面的代码，已知gaussian_kernel 服从正态分布，则 
            torch.exp(noise_sigma*gassian_kernel.sample(noise_sigma.size())) 
    服从对数正态分布，其中我们施加的 mu, sigma 都是对应的正态分布的参数，而非对数正态分布的参数

    又由于 (m, v) 和 (mu, sigma) 是一一映射，我们控制谁都一样，
    所以，为了方便和统一，今后：
    1）需要采样对数正态分布样本，都采取先用 mu, sigma 采样正态分布，再 exp 的方式
    2）需要估计对数正态分布参数，都采取先对对数正态分布样本取 log，再用 norm.fit 拟合的方式
    '''
    
    noise_sigma = mlesigma(f_name)
    gassian_kernel = torch.distributions.Normal(0.0, noise_sigma)
    with torch.no_grad():
        for theta in model.parameters():
            abstheta = torch.abs(theta) # 求参数的绝对值
            normalized_theta = abstheta / (torch.max(abstheta)+1e-8) # 归一化

            theta_index = normalized_theta*(required_len-1)
            theta_index = theta_index.type(torch.LongTensor) # 求各参数对应的下标位置

            noise_index = normalized_theta*100
            noise_index = noise_index.type(torch.LongTensor)
            noise_index[noise_index>=100]=99

            theta_ratio = torch.Tensor(ratio)[theta_index] # theta = theta * (c_real-c_min) / (c_max-c_min)

            # noise_sigma = torch.Tensor(np.log(1+np.square(yfit_std/yfit)))[noise_index]
            # theta.mul_(to_device(theta_ratio * torch.exp(noise_sigma*gassian_kernel.sample(noise_sigma.size())), device))
            # print(noise_sigma.shape)
            theta.mul_(to_device(theta_ratio*torch.exp(gassian_kernel.sample(noise_sigma.size())), device))
            # theta.mul_(to_device(theta_ratio*torch.exp(noise_sigma), device))






# def Kalman1D(observations,damping=0):
#     # To return the smoothed time series data
#     observation_covariance = damping
#     initial_value_guess = observations[0]
#     transition_matrix = 1
#     transition_covariance = 0.1
#     initial_value_guess
#     kf = KalmanFilter(
#             initial_state_mean=initial_value_guess,
#             initial_state_covariance=observation_covariance,
#             observation_covariance=observation_covariance,
#             transition_covariance=transition_covariance,
#             transition_matrices=transition_matrix
#         )
#     pred_state, state_cov = kf.smooth(observations)
#     return pred_state

def LCIS(arr): # 求最长连续[递增]子序列，该[递增]序列中最多允许两次递减
    decrease_cnt = 0
    start_index = 0
    end_index = 0
    sub_len = 0
    longest_start = 0
    longest_end = 0
    longest_len = 0
    decrease_point_0 = -1
    decrease_point_1 = -1
    for i in range(1, len(arr)):
        if arr[i] > arr[i-1]:
            end_index += 1
        else:
            decrease_cnt += 1
            if decrease_cnt == 1:
                end_index += 1
                decrease_point_0 = end_index

            elif decrease_cnt == 2:
                end_index += 1
                decrease_point_1 = end_index
                
            else:
                sub_len = end_index - start_index + 1
                if longest_len < sub_len:
                    longest_len = sub_len
                    longest_start = start_index
                    longest_end = end_index
                start_index = decrease_point_0
                decrease_point_0 = decrease_point_1
                decrease_point_1 = i
                end_index = i
                decrease_cnt = 2

    sub_len = end_index - start_index + 1
    if longest_len < sub_len:
        longest_len = sub_len
        longest_start = start_index
        longest_end = end_index

    return longest_start, longest_end

def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

# def main():
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(4, 3)
    def forward(self, xb):
        xb = xb.view(xb.size(0), -1)
        out = self.linear1(xb)
        return out


model = Net()
# model = weights_mapping(f_name, 0, 1, model=model, device='cpu')
# def weights_mapping(file, mean, std, model, device='cuda'):
# model1 = weights_mapping(f_name, 0, 1, model=model, device='cpu')

# if __name__ == '__main__':
#     main()
# model1
for i in model.parameters():
    print(i)
    break

Parameter containing:
tensor([[ 0.2903, -0.3241,  0.0031, -0.0674],
        [ 0.4362, -0.2060,  0.2714, -0.0670],
        [-0.1672, -0.2732, -0.0159, -0.3084]], requires_grad=True)


In [19]:
f_name = 'I-V_data_25min_AAO_10min_Pb_ED_1h_180C_MAI_memory_8V_2.xlsx'
weights_mapping(f_name, required_len=65, ratio=, yfit, yfit_std, model, device='cuda')

NameError: name 'required_len' is not defined

In [17]:
for i in model.parameters():
    print(i)
    break

Parameter containing:
tensor([[-0.4771,  0.0182, -0.0350,  0.0472],
        [-0.0287, -0.4223, -0.0508, -0.4034],
        [ 0.2235, -0.2301, -0.1098,  0.2376]], requires_grad=True)


In [16]:
model1 = weights_mapping(f_name, 0, 1, model=model, device='cpu')

In [13]:
for i in model.parameters():
    print(i)
    break

Parameter containing:
tensor([[-0.3079,  0.4667,  0.4657,  0.2650],
        [-0.4818, -0.4662, -0.4599,  0.2789],
        [ 0.3211, -0.0410,  0.4333,  0.1952]], requires_grad=True)


In [4]:
def calculate(file):
    df = pd.read_excel(file)
    
    # # df = pd.read_excel('I-V_data_25min_AAO_10min_Pb_ED_1h_180C_MAI_memory_8V_2.xlsx') #0.55
    # # df = pd.read_excel('I-V_data_30min_AAO_5min second etch_15min_Pb_ED_3h_180C_MAI_200nm_Ag_memory_6V.xlsx') #0.33
    # # df = pd.read_excel('./hardware_noise/hardware_data/I-V_data_0.7um_length_200nm_diameter_NA_third_etch_10min_Pb_ED_1h_180C_MAI_no_100nm_Ag_memory_1V_carbon_paste.xlsx') #0.14
    # # df = pd.read_excel('./hardware_noise/I-V_data_25min_AAO_10min_Pb_ED_1h_180C_MAI_memory_8V_2.xlsx') #0.55    
    # df = pd.read_excel('./hardware_noise/I-V_data_30min_AAO_5min second etch_15min_Pb_ED_3h_180C_MAI_200nm_Ag_memory_6V.xlsx') #0.33
    # file = './hardware_noise/I-V_data_30min_AAO_5min second etch_15min_Pb_ED_3h_180C_MAI_200nm_Ag_memory_6V.xlsx'
    # sunqiao/OpenPCDet/tools/hardware_noise/hardware_data/I-V_data_0.7um_length_200nm_diameter_NA_third_etch_10min_Pb_ED_1h_180C_MAI_no_100nm_Ag_memory_1V_carbon_paste.xlsx
    dropcolumn = []
    for i in range(len(df.columns)):
        if 'Unnamed' in df.columns[i]:
            dropcolumn.append(df.columns[i])
            

    df = df.drop(columns=dropcolumn)

    #calculate mean and std current for each row and remove the NAN
    voltage = df['voltage']
    current_mean_list = []
    current_std_list = []
    for i in range(len(voltage)):
        current_row = df.iloc[[i]].to_numpy()
        current_row = current_row[~np.isnan(current_row)]
        
        low_percentile = np.percentile(current_row, 25)
        high_percentile = np.percentile(current_row, 75)
        current_row = current_row[(current_row >=low_percentile ) & (current_row <= high_percentile)]
        
        current_mean = np.mean(current_row)
        
        
        current_std = np.std(current_row)
        current_mean_list.append(current_mean)
        current_std_list.append(current_std)

    current_mean_list = np.array(current_mean_list)
    current_std_list = np.array(current_std_list)

    voltage_list = df['voltage'].to_numpy()

    conductance_mean = current_mean_list/(voltage_list+1e-9)
    conductance_std  = current_std_list/(voltage_list+1e-9)

    c_mean_smooth = conductance_mean[1:100] 

    c_std_smooth = conductance_std[1:100]


    c_mean_smooth = Kalman1D(c_mean_smooth,damping=1)
    c_std_smooth  = Kalman1D(c_std_smooth,damping=1)


    return c_mean_smooth, c_std_smooth